<a href="https://www.kaggle.com/code/rrowwe/ing-hackathon?scriptVersionId=170359468" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from datetime import datetime
import tensorflow_decision_forests as tfdf

2024-04-03 20:50:55.590385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 20:50:55.590475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 20:50:55.591837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
in_time = pd.read_csv('/kaggle/input/in-time/in_time.csv')

In [3]:
in_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310000 entries, 0 to 309999
Columns: 307 entries, Ref_month to Target
dtypes: float64(186), int64(116), object(5)
memory usage: 726.1+ MB


In [4]:
#for each column, displaying the number of rows containing the Null marker value (-9999)
column_counts = {}
for column in in_time.columns:
    column_counts[column] = (in_time[column] == -9999).sum()
    
column_counts

{'Ref_month': 0,
 'Customer_id': 0,
 'Birth_date': 0,
 'No_dependants': 0,
 'Time_in_address': 0,
 'Time_in_current_job': 0,
 'Credit_cards': 0,
 'Debit_cards': 0,
 'Active_accounts': 0,
 'Oldest_account_date': 0,
 'Active_loans': 0,
 'Active_mortgages': 254239,
 'Active_credit_card_lines': 273097,
 'Contract_origination_date': 0,
 'Contract_end_date': 0,
 'Num_borrowers': 0,
 'Current_installment': 0,
 'External_term_loan_balance': 309855,
 'External_mortgage_balance': 309855,
 'External_credit_card_balance': 309855,
 'DPD_term_loan_H12': 0,
 'DPD_term_loan_H11': 0,
 'DPD_term_loan_H10': 0,
 'DPD_term_loan_H9': 0,
 'DPD_term_loan_H8': 0,
 'DPD_term_loan_H7': 0,
 'DPD_term_loan_H6': 0,
 'DPD_term_loan_H5': 0,
 'DPD_term_loan_H4': 0,
 'DPD_term_loan_H3': 0,
 'DPD_term_loan_H2': 0,
 'DPD_term_loan_H1': 0,
 'DPD_term_loan_H0': 0,
 'DPD_credit_card_H12': 0,
 'DPD_credit_card_H11': 0,
 'DPD_credit_card_H10': 0,
 'DPD_credit_card_H9': 0,
 'DPD_credit_card_H8': 0,
 'DPD_credit_card_H7': 0,
 '

In [5]:
#dropping Null-contaminated columns, which cannot be meaningfully imputed (the non-corrupted portion of the dataset is not large enough to be representative of the entire population)
in_time.drop(['External_term_loan_balance', 'External_mortgage_balance', 'External_credit_card_balance'], axis=1, inplace=True)

In [6]:
X = in_time.drop('Target', axis=1)

In [7]:
y = in_time['Target']

In [8]:
def date_to_unix_timestamp(date_string: str, date_format: str='%d-%m-%Y') -> float:
    """
    Function to convert a date string into its unix timestamp form.
    
    Accepts:
    - date_string: the string to format;
    - date_format: the format of the date (e.g.: day/month/year).
    
    Returns:
    - timestamp: the float unix timestamp. 
    
    """
    
    date_object = datetime.strptime(date_string, date_format)
    timestamp = date_object.timestamp()
        
    return float(timestamp)

In [9]:
def ref_month_to_unix_timestamp(date_string: str, date_format: str='%m-%Y', is_ref_month=False) -> float:
    """
    Function to convert a date string into its unix timestamp form. (SPECIFICALLY FOR THE REF_MONTH COLUMN)
    
    Accepts:
    - date_string: the string to format;
    - date_format: the format of the date (e.g.: day/month/year).
    
    Returns:
    - timestamp: the float unix timestamp. 
    
    """
    
    date_object = datetime.strptime(date_string, date_format)
    timestamp = date_object.timestamp()
        
    return float(timestamp)

In [10]:
columns_to_format = ['Birth_date', 'Contract_origination_date', 'Contract_end_date', 'Oldest_account_date']

for col in columns_to_format:
    X[col] = (X[col].apply(date_to_unix_timestamp)).astype(float)

In [11]:
X.head() #making sure the conversion was successful 

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H9,out_transactions_amt_H8,out_transactions_amt_H7,out_transactions_amt_H6,out_transactions_amt_H5,out_transactions_amt_H4,out_transactions_amt_H3,out_transactions_amt_H2,out_transactions_amt_H1,out_transactions_amt_H0
0,11-2022,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,3960.43,4576.57,3981.55,4607.80,4182.93,3831.11,4210.16,3832.29,4727.64,4022.14
1,07-2022,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3552.80,3472.92,3559.04,3564.92,3614.82,3602.57,3552.99,3584.06,3512.58,3624.90
2,01-2022,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,13440.50,14188.19,12669.79,13334.48,12731.75,12651.15,12282.69,12287.79,12827.02,12984.93
3,12-2022,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,4390.50,4286.46,3914.78,4005.07,4094.73,3926.17,4433.92,3964.59,4014.69,4331.89
4,06-2022,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,4439.33,4421.30,4666.98,5053.99,4816.87,4314.71,4711.88,5014.49,4152.09,5288.72


In [12]:
X['Ref_month'] = (X['Ref_month'].apply(ref_month_to_unix_timestamp)).astype(float)

In [13]:
X['Ref_month'].dtype

dtype('float64')

In [14]:
X['Ref_month'] #making sure the conversion was successful 

0         1.667261e+09
1         1.656634e+09
2         1.640995e+09
3         1.669853e+09
4         1.654042e+09
              ...     
309995    1.646093e+09
309996    1.664582e+09
309997    1.651363e+09
309998    1.656634e+09
309999    1.667261e+09
Name: Ref_month, Length: 310000, dtype: float64

In [15]:
X.replace(-9999, 0, inplace=True)

In [16]:
in_time.dtypes

Ref_month                   object
Customer_id                  int64
Birth_date                  object
No_dependants                int64
Time_in_address            float64
                            ...   
out_transactions_amt_H3    float64
out_transactions_amt_H2    float64
out_transactions_amt_H1    float64
out_transactions_amt_H0    float64
Target                       int64
Length: 304, dtype: object

In [17]:
#columns_with_negative_values = X.columns[(X < 0).any()]
#print(columns_with_negative_values)

#negative_value_counts = {}

# Iterate over each column
#for column in columns_with_negative_values:
    # Count the number of negative values in the column
 #   negative_count = (in_time[column] < 0).sum()
    
    # Store the count in the dictionary
  #  negative_value_counts[column] = negative_count
    
#negative_value_counts

#  **CHECKING VARIABLE DISTRIBUTIONS**

In [18]:
def compare_overdue_terms(df):
    """
    Compare 'Overdue_term_loan_Hx' with 'Overdue_term_loan_H(x-1)' and add 1 to counter if the value in 'Overdue_term_loan_H(x-1)' 
    is lower compared to 'Overdue_term_loan_Hx'.
    """

    df['Overdue_counter_term_loan'] = 0

    df['Overdue_counter_term_loan'] += (df['Overdue_term_loan_H3'] > df['Overdue_term_loan_H4']).astype(int)
    df['Overdue_counter_term_loan'] += (df['Overdue_term_loan_H2'] > df['Overdue_term_loan_H3']).astype(int)
    df['Overdue_counter_term_loan'] += (df['Overdue_term_loan_H1'] > df['Overdue_term_loan_H2']).astype(int)
    df['Overdue_counter_term_loan'] += (df['Overdue_term_loan_H0'] > df['Overdue_term_loan_H1']).astype(int)
    
    
    df = df.drop(columns=[col for col in df.columns if 'Overdue_term_loan_H' in col], inplace=True)

    
    return df

In [19]:
def compare_overdue_card(df):
    """
    Compare 'Overdue_credit_card_Hx' with 'Overdue_credit_card_H(x-1)' and add 1 to counter if the value in 'Overdue_credit_card_H(x-1)' 
    is lower compared to 'Overdue_credit_card_Hxx'.
    """
   
    df['Overdue_counter_credit_card'] = 0
    
    df['Overdue_counter_credit_card'] += (df['Overdue_credit_card_H3'] > df['Overdue_credit_card_H4']).astype(int)
    df['Overdue_counter_credit_card'] += (df['Overdue_credit_card_H2'] > df['Overdue_credit_card_H3']).astype(int)
    df['Overdue_counter_credit_card'] += (df['Overdue_credit_card_H1'] > df['Overdue_credit_card_H2']).astype(int)
    df['Overdue_counter_credit_card'] += (df['Overdue_credit_card_H0'] > df['Overdue_credit_card_H1']).astype(int)
        
    df = df.drop(columns=[col for col in df.columns if 'Overdue_credit_card' in col], inplace=True)    
        
    return df

In [20]:
compare_overdue_card(in_time)
compare_overdue_card(X)

In [21]:
def compare_overdue_morgage(df):
    """
    Compare 'Overdue_mortgage_Hx' with 'Overdue_mortgage_H(x-1)' and add 1 to counter if the value in 'Overdue_mortgage_H(x-1)' 
    is lower compared to 'Overdue_mortgage_Hx'.
    """
   
    df['Overdue_counter_mortgage'] = 0
    
    df['Overdue_counter_mortgage'] += (df['Overdue_mortgage_H3'] > df['Overdue_mortgage_H4']).astype(int)
    df['Overdue_counter_mortgage'] += (df['Overdue_mortgage_H2'] > df['Overdue_mortgage_H3']).astype(int)
    df['Overdue_counter_mortgage'] += (df['Overdue_mortgage_H1'] > df['Overdue_mortgage_H2']).astype(int)
    df['Overdue_counter_mortgage'] += (df['Overdue_mortgage_H0'] > df['Overdue_mortgage_H1']).astype(int)
    
    df.drop(columns=[col for col in df.columns if 'Overdue_mortgage_H' in col], inplace=True)
        
    return df

In [22]:
compare_overdue_morgage(in_time)
compare_overdue_morgage(X)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H7,out_transactions_amt_H6,out_transactions_amt_H5,out_transactions_amt_H4,out_transactions_amt_H3,out_transactions_amt_H2,out_transactions_amt_H1,out_transactions_amt_H0,Overdue_counter_credit_card,Overdue_counter_mortgage
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,3981.55,4607.80,4182.93,3831.11,4210.16,3832.29,4727.64,4022.14,0,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3559.04,3564.92,3614.82,3602.57,3552.99,3584.06,3512.58,3624.90,0,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,12669.79,13334.48,12731.75,12651.15,12282.69,12287.79,12827.02,12984.93,0,1
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,3914.78,4005.07,4094.73,3926.17,4433.92,3964.59,4014.69,4331.89,0,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,4666.98,5053.99,4816.87,4314.71,4711.88,5014.49,4152.09,5288.72,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,4398.42,4115.42,4334.34,4113.10,4423.33,4726.98,4200.09,4099.52,0,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,4325.59,4299.23,3834.01,3677.22,3835.20,4658.65,3992.09,4061.19,0,0
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,5325.48,4832.33,5319.56,5117.83,4823.07,5510.92,5145.15,4537.41,0,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,4319.86,4890.04,4619.89,5206.99,4425.69,4695.32,4632.36,4729.46,0,0


In [23]:
type(in_time)

pandas.core.frame.DataFrame

In [24]:
def compare_os_term_terms(df):

    df['Os_term_loan'] = 0

    df['Os_term_loan'] += (df['Os_term_loan_H3'] > df['Os_term_loan_H4']).astype(int)
    df['Os_term_loan'] += (df['Os_term_loan_H2'] > df['Os_term_loan_H3']).astype(int)
    df['Os_term_loan'] += (df['Os_term_loan_H1'] > df['Os_term_loan_H2']).astype(int)
    df['Os_term_loan'] += (df['Os_term_loan_H0'] > df['Os_term_loan_H1']).astype(int)
    
    
    #df.drop(['Overdue_counter_term_loan_H0', 'Overdue_counter_term_loan_H1', 'Overdue_counter_term_loan_H2', 'Overdue_counter_term_loan_H3', 'Overdue_counter_term_loan_H4', 'Overdue_counter_term_loan_H5', 'Overdue_counter_term_loan_H6', 'Overdue_counter_term_loan_H7', 'Overdue_counter_term_loan_H7', 'Overdue_counter_term_loan_H8', 'Overdue_counter_term_loan_H9', 'Overdue_counter_term_loan_H10', 'Overdue_counter_term_loan_H11', 'Overdue_counter_term_loan_H12'], axis=1, inplace=True)

    
    return df

In [25]:
def compare_os_credit_card_terms(df):

    df['Os_credit_card'] = 0

    df['Os_credit_card'] += (df['Os_credit_card_H3'] > df['Os_credit_card_H4']).astype(int)
    df['Os_credit_card'] += (df['Os_credit_card_H2'] > df['Os_credit_card_H3']).astype(int)
    df['Os_credit_card'] += (df['Os_credit_card_H1'] > df['Os_credit_card_H2']).astype(int)
    df['Os_credit_card'] += (df['Os_credit_card_H0'] > df['Os_credit_card_H1']).astype(int)
    
    
    #df.drop(['Overdue_counter_term_loan_H0', 'Overdue_counter_term_loan_H1', 'Overdue_counter_term_loan_H2', 'Overdue_counter_term_loan_H3', 'Overdue_counter_term_loan_H4', 'Overdue_counter_term_loan_H5', 'Overdue_counter_term_loan_H6', 'Overdue_counter_term_loan_H7', 'Overdue_counter_term_loan_H7', 'Overdue_counter_term_loan_H8', 'Overdue_counter_term_loan_H9', 'Overdue_counter_term_loan_H10', 'Overdue_counter_term_loan_H11', 'Overdue_counter_term_loan_H12'], axis=1, inplace=True)

    
    return df

In [26]:
def compare_os_mortgage_terms(df):

    df['Os_mortgage'] = 0

    df['Os_mortgage'] += (df['Os_mortgage_H3'] > df['Os_mortgage_H4']).astype(int)
    df['Os_mortgage'] += (df['Os_mortgage_H2'] > df['Os_mortgage_H3']).astype(int)
    df['Os_mortgage'] += (df['Os_mortgage_H1'] > df['Os_mortgage_H2']).astype(int)
    df['Os_mortgage'] += (df['Os_mortgage_H0'] > df['Os_mortgage_H1']).astype(int)
    
    
    #df.drop(['Overdue_counter_term_loan_H0', 'Overdue_counter_term_loan_H1', 'Overdue_counter_term_loan_H2', 'Overdue_counter_term_loan_H3', 'Overdue_counter_term_loan_H4', 'Overdue_counter_term_loan_H5', 'Overdue_counter_term_loan_H6', 'Overdue_counter_term_loan_H7', 'Overdue_counter_term_loan_H7', 'Overdue_counter_term_loan_H8', 'Overdue_counter_term_loan_H9', 'Overdue_counter_term_loan_H10', 'Overdue_counter_term_loan_H11', 'Overdue_counter_term_loan_H12'], axis=1, inplace=True)

    
    return df

In [27]:
compare_os_credit_card_terms(in_time)
compare_os_credit_card_terms(X)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H6,out_transactions_amt_H5,out_transactions_amt_H4,out_transactions_amt_H3,out_transactions_amt_H2,out_transactions_amt_H1,out_transactions_amt_H0,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,4607.80,4182.93,3831.11,4210.16,3832.29,4727.64,4022.14,0,0,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3564.92,3614.82,3602.57,3552.99,3584.06,3512.58,3624.90,0,0,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,13334.48,12731.75,12651.15,12282.69,12287.79,12827.02,12984.93,0,1,1
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,4005.07,4094.73,3926.17,4433.92,3964.59,4014.69,4331.89,0,0,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,5053.99,4816.87,4314.71,4711.88,5014.49,4152.09,5288.72,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,4115.42,4334.34,4113.10,4423.33,4726.98,4200.09,4099.52,0,0,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,4299.23,3834.01,3677.22,3835.20,4658.65,3992.09,4061.19,0,0,0
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,4832.33,5319.56,5117.83,4823.07,5510.92,5145.15,4537.41,0,0,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,4890.04,4619.89,5206.99,4425.69,4695.32,4632.36,4729.46,0,0,0


In [28]:
compare_os_mortgage_terms(in_time)
compare_os_mortgage_terms(X)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H5,out_transactions_amt_H4,out_transactions_amt_H3,out_transactions_amt_H2,out_transactions_amt_H1,out_transactions_amt_H0,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card,Os_mortgage
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,4182.93,3831.11,4210.16,3832.29,4727.64,4022.14,0,0,0,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3614.82,3602.57,3552.99,3584.06,3512.58,3624.90,0,0,0,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,12731.75,12651.15,12282.69,12287.79,12827.02,12984.93,0,1,1,1
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,4094.73,3926.17,4433.92,3964.59,4014.69,4331.89,0,0,0,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,4816.87,4314.71,4711.88,5014.49,4152.09,5288.72,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,4334.34,4113.10,4423.33,4726.98,4200.09,4099.52,0,0,0,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,3834.01,3677.22,3835.20,4658.65,3992.09,4061.19,0,0,0,0
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,5319.56,5117.83,4823.07,5510.92,5145.15,4537.41,0,0,0,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,4619.89,5206.99,4425.69,4695.32,4632.36,4729.46,0,0,0,0


In [29]:
compare_os_term_terms(in_time)
compare_os_term_terms(X)    

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H4,out_transactions_amt_H3,out_transactions_amt_H2,out_transactions_amt_H1,out_transactions_amt_H0,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card,Os_mortgage,Os_term_loan
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,3831.11,4210.16,3832.29,4727.64,4022.14,0,0,0,0,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3602.57,3552.99,3584.06,3512.58,3624.90,0,0,0,0,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,12651.15,12282.69,12287.79,12827.02,12984.93,0,1,1,1,1
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,3926.17,4433.92,3964.59,4014.69,4331.89,0,0,0,0,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,4314.71,4711.88,5014.49,4152.09,5288.72,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,4113.10,4423.33,4726.98,4200.09,4099.52,0,0,0,0,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,3677.22,3835.20,4658.65,3992.09,4061.19,0,0,0,0,1
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,5117.83,4823.07,5510.92,5145.15,4537.41,0,0,0,0,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,5206.99,4425.69,4695.32,4632.36,4729.46,0,0,0,0,0


In [30]:
compare_overdue_terms(in_time)
compare_overdue_terms(X)    

In [31]:
def default_flag_counter(df):
    
    df['default_flag_counter'] = 0
    
    df['default_flag_counter'] += df['Default_flag_H0']
    df['default_flag_counter'] += df['Default_flag_H1']
    df['default_flag_counter'] += df['Default_flag_H2']
    df['default_flag_counter'] += df['Default_flag_H3']
    df['default_flag_counter'] += df['Default_flag_H4']
    
    df.drop(columns=[col for col in df.columns if 'Default' in col], inplace=True)
    
    return df

In [32]:
def dpd_feature(df, value=30):
    """
    Adds a feature based on the specified value and DPD columns.

    Parameters:
        df (DataFrame): The DataFrame containing the DPD columns.
        value (int): The value to check against for each DPD column.
    """
    # List of DPD columns
    dpd_columns = ['DPD_term_loan_H0', 'DPD_term_loan_H1', 'DPD_term_loan_H2', 'DPD_term_loan_H3', 'DPD_term_loan_H4',
                   'DPD_credit_card_H0', 'DPD_credit_card_H1', 'DPD_credit_card_H2', 'DPD_credit_card_H3', 'DPD_credit_card_H4',
                   'DPD_mortgage_H0', 'DPD_mortgage_H1', 'DPD_mortgage_H2', 'DPD_mortgage_H3', 'DPD_mortgage_H4']

#Create a new feature with initial value 0
    df['Dpd_above_threshold'] = 0

# Iterate over each DPD column and update the new feature if the condition is met
    for column in dpd_columns:
        df['Dpd_above_threshold'] |= df[column].apply(lambda x: 1 if x <= value else 0)
    df.drop(columns=[col for col in df.columns if 'DPD' in col], inplace=True)

In [33]:
dpd_feature(in_time)
dpd_feature(X)

In [34]:
'DPD_term_loan_H0' in in_time.columns

False

In [35]:
def add_income_volatility_feature(df, value=0.5):
    """
    Add a new feature 'Income_volatility' based on the specified value and income columns.

    Parameters:
        df (DataFrame): The DataFrame containing the income columns.
        value (float): The percentage value to compare against (default is 0.5).
    """ 
    # Create a new feature 'Income_volatility' with initial value 0
    df['Income_volatility'] = 0

#Set 'Income_volatility' to 1 if any income column is <= Income_H4 * value
    df.loc[(df['Income_H0'] <= df['Income_H4'] * value) | 
           (df['Income_H1'] <= df['Income_H4'] * value) | 
           (df['Income_H2'] <= df['Income_H4'] * value) | 
           (df['Income_H3'] <= df['Income_H4'] * value), 'Income_volatility'] = 1

    df.drop(columns=[col for col in df.columns if 'Income_H' in col], inplace=True)

    return df

In [36]:
add_income_volatility_feature(in_time)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H0,Target,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card,Os_mortgage,Os_term_loan,Overdue_counter_term_loan,Dpd_above_threshold,Income_volatility
0,11-2022,33344475,21-05-1988,0,3.41,4.13,0,2,2,26-06-2008,...,4022.14,0,0,0,0,0,0,0,1,0
1,07-2022,38290809,07-07-1983,1,2.28,11.00,0,1,1,16-05-2015,...,3624.90,0,0,0,0,0,0,0,1,0
2,01-2022,33178125,16-09-1982,1,8.99,1.73,1,2,1,11-02-2005,...,12984.93,0,0,1,1,1,1,1,1,0
3,12-2022,37129367,13-05-1985,2,5.55,1.76,0,1,1,27-01-2006,...,4331.89,0,0,0,0,0,0,0,1,0
4,06-2022,31904710,26-03-1989,1,4.48,8.43,1,1,1,28-06-2008,...,5288.72,0,0,0,3,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,03-2022,31104928,13-09-1989,1,1.92,0.63,0,1,1,22-02-2010,...,4099.52,0,0,0,0,0,0,0,1,0
309996,10-2022,38699158,09-10-1973,1,19.89,3.03,0,1,2,30-04-2007,...,4061.19,0,0,0,0,0,1,1,1,0
309997,05-2022,37435274,12-09-1986,3,3.92,3.90,0,2,1,23-08-2015,...,4537.41,0,0,0,0,0,0,0,1,0
309998,07-2022,36145194,11-07-1972,1,18.38,14.47,0,1,1,28-09-2004,...,4729.46,0,0,0,0,0,0,0,1,0


In [37]:
add_income_volatility_feature(X)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H1,out_transactions_amt_H0,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card,Os_mortgage,Os_term_loan,Overdue_counter_term_loan,Dpd_above_threshold,Income_volatility
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,4727.64,4022.14,0,0,0,0,0,0,1,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3512.58,3624.90,0,0,0,0,0,0,1,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,12827.02,12984.93,0,1,1,1,1,1,1,0
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,4014.69,4331.89,0,0,0,0,0,0,1,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,4152.09,5288.72,0,0,3,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,4200.09,4099.52,0,0,0,0,0,0,1,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,3992.09,4061.19,0,0,0,0,1,1,1,0
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,5145.15,4537.41,0,0,0,0,0,0,1,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,4632.36,4729.46,0,0,0,0,0,0,1,0


In [38]:
default_flag_counter(in_time)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,Target,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card,Os_mortgage,Os_term_loan,Overdue_counter_term_loan,Dpd_above_threshold,Income_volatility,default_flag_counter
0,11-2022,33344475,21-05-1988,0,3.41,4.13,0,2,2,26-06-2008,...,0,0,0,0,0,0,0,1,0,0
1,07-2022,38290809,07-07-1983,1,2.28,11.00,0,1,1,16-05-2015,...,0,0,0,0,0,0,0,1,0,0
2,01-2022,33178125,16-09-1982,1,8.99,1.73,1,2,1,11-02-2005,...,0,0,1,1,1,1,1,1,0,0
3,12-2022,37129367,13-05-1985,2,5.55,1.76,0,1,1,27-01-2006,...,0,0,0,0,0,0,0,1,0,0
4,06-2022,31904710,26-03-1989,1,4.48,8.43,1,1,1,28-06-2008,...,0,0,0,3,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,03-2022,31104928,13-09-1989,1,1.92,0.63,0,1,1,22-02-2010,...,0,0,0,0,0,0,0,1,0,0
309996,10-2022,38699158,09-10-1973,1,19.89,3.03,0,1,2,30-04-2007,...,0,0,0,0,0,1,1,1,0,0
309997,05-2022,37435274,12-09-1986,3,3.92,3.90,0,2,1,23-08-2015,...,0,0,0,0,0,0,0,1,0,0
309998,07-2022,36145194,11-07-1972,1,18.38,14.47,0,1,1,28-09-2004,...,0,0,0,0,0,0,0,1,0,0


In [39]:
default_flag_counter(X)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,out_transactions_amt_H0,Overdue_counter_credit_card,Overdue_counter_mortgage,Os_credit_card,Os_mortgage,Os_term_loan,Overdue_counter_term_loan,Dpd_above_threshold,Income_volatility,default_flag_counter
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,4022.14,0,0,0,0,0,0,1,0,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,3624.90,0,0,0,0,0,0,1,0,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,12984.93,0,1,1,1,1,1,1,0,0
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,4331.89,0,0,0,0,0,0,1,0,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,5288.72,0,0,3,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,4099.52,0,0,0,0,0,0,1,0,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,4061.19,0,0,0,0,1,1,1,0,0
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,4537.41,0,0,0,0,0,0,1,0,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,4729.46,0,0,0,0,0,0,1,0,0


In [40]:
in_time = in_time.drop(['Birth_date', 'Contract_origination_date', 'Contract_end_date', 'Oldest_account_date', 'Ref_month'], axis=1)

In [41]:
def accounts_feature(df, threshold=0.5):
    """
    Adds a new feature indicating whether the values in the specified columns decrease in a majority of columns.

    Parameters:
        df (DataFrame): The DataFrame containing the columns.
        threshold (float): The threshold of columns showing a decrease to consider it as a majority.
    """

    columns_to_check_sav = ['Savings_amount_balance_H4',
                    'Savings_amount_balance_H3', 'Savings_amount_balance_H2',
                    'Savings_amount_balance_H1', 'Savings_amount_balance_H0']

#Calculate the differences between consecutive values for each row
    diffs_sav = df[columns_to_check_sav].diff(axis=1)

#Count the number of columns where the difference is negative for each row
    decrease_count_sav = (diffs_sav < 0).sum(axis=1)

#Check if the decrease count is greater than the threshold
    decrease_mask_sav = (decrease_count_sav / len(columns_to_check_sav)) > threshold

#Create a new feature with 1 for rows where a majority of values decrease, 0 otherwise
    df['Saving_acc_decrease'] = decrease_mask_sav.astype(int)

    columns_to_check_cur = ['Current_amount_balance_H4',
       'Current_amount_balance_H3', 'Current_amount_balance_H2',
       'Current_amount_balance_H1', 'Current_amount_balance_H0',]

#Calculate the differences between consecutive values for each row
    diffs_cur = df[columns_to_check_cur].diff(axis=1)

#Check if all differences are negative for each row
    decrease_mask_cur = (diffs_cur < 0).all(axis=1)

#Create a new feature with 1 for rows where values decrease overall, 0 otherwise
    df['Current_acc_decrease'] = decrease_mask_cur.astype(int)

    df.drop(columns=[col for col in df.columns if 'Savings_amount_balance_H' in col], inplace=True)
    df.drop(columns=[col for col in df.columns if 'Current_amount_balance_H' in col], inplace=True)

    return df

In [54]:
def more_incoming_transactions(df):
    inc_transactions_amt_cols = ['inc_transactions_amt_H1', 'inc_transactions_amt_H2', 'inc_transactions_amt_H3', 'inc_transactions_amt_H4']
    inc_transactions_amt_sum = df['inc_transactions_amt_H0']
    for col in inc_transactions_amt_cols:
        inc_transactions_amt_sum += df[col]
    out_transactions_amt_cols = ['out_transactions_amt_H1', 'out_transactions_amt_H2', 'out_transactions_amt_H3', 'out_transactions_amt_H4']
    out_transactions_amt_sum = df['out_transactions_amt_H0']
    for col in out_transactions_amt_cols:
        out_transactions_amt_sum += df[col]
    df['spending_amount_on_transactions_amt_sum'] = inc_transactions_amt_sum - out_transactions_amt_sum
    df['more_spending_amount_on_transactions'] = df['spending_amount_on_transactions_amt_sum'].apply(lambda x: 1 if x > 0 else 0)
    df.drop(columns=[col for col in df.columns if 'inc_transactions_amt_H' in col], inplace=True)
    df.drop(columns=[col for col in df.columns if 'out_transactions_amt_H' in col], inplace=True)
    return df

In [55]:
in_time = more_incoming_transactions(in_time)
X = more_incoming_transactions(X)

In [42]:
accounts_feature(in_time)
accounts_feature(X)

,Ref_month,Customer_id,Birth_date,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Oldest_account_date,...,Overdue_counter_mortgage,Os_credit_card,Os_mortgage,Os_term_loan,Overdue_counter_term_loan,Dpd_above_threshold,Income_volatility,default_flag_counter,Saving_acc_decrease,Current_acc_decrease
0,1.667261e+09,33344475,580176000.0,0,3.41,4.13,0,2,2,1.214438e+09,...,0,0,0,0,0,1,0,0,0,0
1,1.656634e+09,38290809,426384000.0,1,2.28,11.00,0,1,1,1.431734e+09,...,0,0,0,0,0,1,0,0,1,0
2,1.640995e+09,33178125,400982400.0,1,8.99,1.73,1,2,1,1.108080e+09,...,1,1,1,1,1,1,0,0,0,0
3,1.669853e+09,37129367,484790400.0,2,5.55,1.76,0,1,1,1.138320e+09,...,0,0,0,0,0,1,0,0,0,0
4,1.654042e+09,31904710,606873600.0,1,4.48,8.43,1,1,1,1.214611e+09,...,0,3,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,1.646093e+09,31104928,621648000.0,1,1.92,0.63,0,1,1,1.266797e+09,...,0,0,0,0,0,1,0,0,1,0
309996,1.664582e+09,38699158,118972800.0,1,19.89,3.03,0,1,2,1.177891e+09,...,0,0,0,1,1,1,0,0,0,0
309997,1.651363e+09,37435274,526867200.0,3,3.92,3.90,0,2,1,1.440288e+09,...,0,0,0,0,0,1,0,0,0,0
309998,1.656634e+09,36145194,79660800.0,1,18.38,14.47,0,1,1,1.096330e+09,...,0,0,0,0,0,1,0,0,1,0


In [56]:
select = SelectKBest(f_classif, k=9)
X_new = select.fit_transform(X, y)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 30  95 147 153 157] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [57]:
filter = select.get_support()
features = np.array(X.columns)
 
print("Selected best 9 features:")
print(features[filter])

Selected best 9 features:
['utilized_limit_in_revolving_loans_H4'
 'utilized_limit_in_revolving_loans_H3'
 'utilized_limit_in_revolving_loans_H2'
 'utilized_limit_in_revolving_loans_H1'
 'utilized_limit_in_revolving_loans_H0' 'Os_term_loan'
 'Overdue_counter_term_loan' 'default_flag_counter'
 'spending_amount_on_transactions_amt_sum']


In [45]:
'Default_flag_counter' in in_time.columns

False

In [61]:
in_time_new = in_time[['Saving_acc_decrease','spending_amount_on_transactions_amt_sum',
 'Current_acc_decrease',
 'Income_volatility', 'Dpd_above_threshold', 'Os_term_loan',
 'Overdue_counter_term_loan', 'default_flag_counter', 'Overdue_counter_credit_card', 'Target']]

In [62]:
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(in_time_new, label="Target")

model = tfdf.keras.RandomForestModel()
model.fit(tf_dataset)

print(model.summary())

Use /tmp/tmpoaqls870 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.393276. Found 310000 examples.
Training model...


[INFO 24-04-03 20:59:34.4698 UTC kernel.cc:1233] Loading model from path /tmp/tmpoaqls870/model/ with prefix 53e1908d1df64a17


Model trained in 0:01:28.229272
Compiling model...


[INFO 24-04-03 20:59:35.8084 UTC decision_forest.cc:660] Model loaded with 300 root(s), 641098 node(s), and 6 input feature(s).
[INFO 24-04-03 20:59:35.8084 UTC abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-04-03 20:59:35.8084 UTC kernel.cc:1061] Use fast generic engine


Model compiled.
Model: "random_forest_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (9):
	Current_acc_decrease
	Dpd_above_threshold
	Income_volatility
	Os_term_loan
	Overdue_counter_credit_card
	Overdue_counter_term_loan
	Saving_acc_decrease
	default_flag_counter
	spending_amount_on_transactions_amt_sum

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.                    "default_flag_counter"  0.679114 ################
    2. "spending_amount_on_transactions_amt_sum"  0.455660 ##########
    3.               "Overdue_counter_term_loan"  0.172663 ##
    4.                            "Os_term_loan"  0.110764 
    5.                       "Income_vol